In [ ]:
# parallel computing
using Distributed
addprocs(...) # set number depending on number of cores available on your computer

In [ ]:
# needed packages
@everywhere begin
    using Distributions
    using Bootstrap
    using Statistics
    using LinearAlgebra
    using SharedArrays
end

using DataFrames
using HypothesisTests
using DelimitedFiles
using StatsBase
using Colors
using Gadfly
using Compose

In [ ]:
# set parameters, define priors, etc.
@everywhere begin
    const numb_hyp = 11
    const numb_toss = 500
    const prior = fill(Float32(1/numb_hyp), numb_hyp)
    const likelihood_heads = range(0f0, stop=1, length=numb_hyp)
    const likelihood_tails = range(1f0, stop=0, length=numb_hyp)
end

In [ ]:
@everywhere datFunc(bias) = rand(Bernoulli(bias), numb_toss)

In [ ]:
# Bayes' rule
@everywhere function b_upd(probs::Array{Float32,1}, dat::Array{Bool,1}, toss_num::Int64)
    if dat[toss_num] == true
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end
end

In [ ]:
# EXPL
@everywhere function expl_upd(probs::Array{Float32,1}, dat::Array{Bool,1}, toss_num::Int64, bonus::Float32=0.1)
    val::Float32 = mean(dat[1:toss_num]) * 10 + 1
    vec::Array{Float32,1} = if dat[toss_num] == true
            @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
        else
            @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
        end

    if val % 1 == .5
        vec[floor(Int64, val)] += .5*bonus
        vec[ceil(Int64, val)] += .5*bonus
    else
        vec[round(Int64, val, RoundNearestTiesAway)] += bonus
    end

    return vec / (1 + bonus)
end

In [ ]:
# Good's rule
@everywhere function good_bonus(probs::Array{Float32,1}, res::Bool, λ=2) # with λ=2, we obtain the rule L2 from Douven and Schupbach, Frontiers ...

    pE::Float32 = res == true ? dot(probs, likelihood_heads) : dot(probs, likelihood_tails)
    gb::Array{Float32,1} = res == true ? log.(likelihood_heads ./ pE) : log.(likelihood_tails ./ pE)

    function rsc(i)
        if i >= 0
            1 - exp(2λ^2 * -i^2)
        else
            -1 + exp(2λ^2 * -i^2)
        end
    end

    return map(rsc, gb)

end

@everywhere function good_upd(probs::Array{Float32,1}, dat::Array{Bool,1}, toss_num::Int64, γ::Float32=0.1)

    res::Bool = dat[toss_num]

    probvec::Array{Float32,1} = if res == true
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end

    goodvec::Array{Float32,1} = probvec + γ .* (probvec .* good_bonus(probs, res))

    return goodvec / sum(goodvec)

end

In [ ]:
# Popper's rule
@everywhere function pop_bonus(probs::Array{Float32,1}, res::Bool)

    pE::Float32 = res == true ? dot(probs, likelihood_heads) : dot(probs, likelihood_tails)
    pb::Array{Float32, 1} = res == true ? (likelihood_heads .- pE) ./ (likelihood_heads .+ pE) : (likelihood_tails .- pE) ./ (likelihood_tails .+ pE)

 end

@everywhere function pop_upd(probs::Array{Float32,1}, dat::Array{Bool, 1}, toss_num::Int64, γ::Float32=0.1)

    res::Bool = dat[toss_num]

    probvec::Array{Float32,1} = if res == true
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end

    popvec::Array{Float32,1} = probvec + γ .* (probvec .* pop_bonus(probs, res))

    return popvec / sum(popvec)

end

In [ ]:
@everywhere const numb_agents = 200
@everywhere const numb_generations = 250

In [ ]:
# starting position: 50 Bayesians, and 50 agents per other group (EXPL, Good's rule, Popper's rule), with varying values for c (varying between 0 and 0.25)
groupID = repeat(1.0:4.0, inner=div(numb_agents, 4))
population_start = vcat(fill(0, div(numb_agents, 4)), rand(Uniform(0, .25), 3*div(numb_agents, 4)))
pop_start = Array{Float32,2}(hcat(groupID, population_start));

In [ ]:
@everywhere function survWei(upds::Array{Float32,2}, # modeling probability of death, based on Weibull distribution
                             hyp::Int64,
                             a::Float64,
                             b::Float64,
                             shape::Float64=rand(Uniform(.5, 5)),
                             scale::Float64=rand(Uniform(50, 250)),
                             thresh::Float64=.9)

    t = something(findfirst(upds .> thresh), (numb_toss, 0)) # where in the matrix with probability updates do we find the first value above thresh?
    c = t[2]
    p = t[1]

    # cdf(Weibull(shape, scale), p) below gives the probability of death at the relevant time

    if c == hyp
        1 - (cdf(Weibull(shape, scale), p) / a) # probability goes down if right intervention is made (which is made when the truth is assigned a probability above thresh)
    elseif c == 0
        1 - cdf(Weibull(shape, scale), numb_toss + 1) # if no intervention is made, output survival probability at last time step
    else
        (1 - cdf(Weibull(shape, scale), p)) / b # probability goes down if wrong intervention is made (which happens if a false hypothesis is assigned a probabilty above thresh)
    end
end

In [ ]:
@everywhere function survGam(upds::Array{Float32,2}, # modeling probability of death, based on Gamma distribution
                             hyp::Int64,
                             a::Float64,
                             b::Float64,
                             shape::Float64=rand(Uniform(10, 16)),
                             scale::Float64=rand(Uniform(10, 16)),
                             thresh::Float64=.9)

    t = something(findfirst(upds .> thresh), (numb_toss, 0)) # where in the matrix with probability updates do we find the first value above thresh?
    c = t[2]
    p = t[1]

    if c == hyp
        1 - (cdf(Gamma(shape, scale), p) / a) # the probability goes down if the right intervention is made (and the right intervention is made if the truth is assigned a probability above thresh)
    elseif c == 0
        1 - cdf(Gamma(shape, scale), numb_toss + 1) # if no intervention is made, output survival probability at last time step
    else
        (1 - cdf(Gamma(shape, scale), p)) / b # the probability goes down if the wrong intervention is made (which happens if a false hypothesis is assigned a probabilty above thresh)
    end
end

In [ ]:
@everywhere function patient(rule_index::Float32, c_value::Float32, dist::Function)

    rand_hyp::Int64 = rand(1:11) # pick α hypothesis ("what's wrong with the patient")
    right = rand(Uniform(1, 10)) # effect of right intervention
    wrong = rand(Uniform(1, 10)) # effect of wrong intervention

    data::Array{Bool,1} = datFunc((rand_hyp - 1) / (numb_hyp - 1)) # generate synthetic data for this pick (the test results for the patient)

    updates = Array{Float32,2}(undef, numb_toss + 1, numb_hyp) # initialize array for probabilities

    updates[1, :] = prior # set prior

    if rule_index == 1.0f0
        @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = b_upd(updates[t, :], data, t)
        end
    elseif rule_index == 2.0f0
        @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = expl_upd(updates[t, :], data, t, c_value)
        end
    elseif rule_index == 3.0f0
        @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = good_upd(updates[t, :], data, t, c_value)
        end
    else
        @inbounds for t in 1:numb_toss # generate updates
            updates[t + 1, :] = pop_upd(updates[t, :], data, t, c_value)
        end
    end

    return dist(updates, rand_hyp, right, wrong)
end

In [ ]:
#= tests doctor on 100 patients and calculates average survival score obtained by doctor, so
average probability that patients would survive =#
@everywhere function avScore(rule_index::Float32, c_value::Float32, dist::Function)
    tot = @distributed (+) for i in 1:100
        patient(rule_index, c_value, dist)
    end
    return tot / 100
end

In [ ]:
#= tests all doctors in a population and selects the 50 percent fittest; outputs those
as well as a copy of each of them =#
function population_upd_rep(pop::Array{Float32,2}, dist::Function)
    agent_scores = SharedArray{Float32,1}(numb_agents)
    @sync @distributed for i in 1:numb_agents
        @inbounds agent_scores[i] = avScore(pop[i, :]..., dist)
    end
    best_index = findall(agent_scores .>= Statistics.median(agent_scores))
    best = pop[best_index[1:Int(numb_agents/2)], :]
    return vcat(best, best), agent_scores
end

In [ ]:
run(`mkdir data`)

In [ ]:
const numb_simulations = 50

In [ ]:
function sim_run(dist::Function)
    k = 1
    while k < numb_simulations + 1
        groupID = repeat(1.0:4.0, inner=div(numb_agents, 4))
        population_start = vcat(fill(0, div(numb_agents, 4)), rand(Uniform(0, .25), 3*div(numb_agents, 4)))
        pop_start = Array{Float32,2}(hcat(groupID, population_start))
        pop_upd_c_a = Array{Float32,3}(undef, numb_agents, 2, numb_generations + 1)
        pop_upd_f = Array{Float32,2}(undef, numb_agents, numb_generations + 1)
        pop_upd_c_a[:, :, 1] = pop_start

        @inbounds for i in 1:numb_generations
            pop_upd_c_a[:, :, i + 1], pop_upd_f[:, i] = population_upd_rep(pop_upd_c_a[:, :, i], dist)
        end

        @inbounds for i in 1:numb_agents
            pop_upd_f[i, numb_generations + 1] = avScore(pop_upd_c_a[i, 1:2, numb_generations + 1]..., dist)
        end

        res_a = Array{Int32,2}(undef, numb_agents, numb_generations + 1)
        res_c = Array{Float32,2}(undef, numb_agents, numb_generations + 1)

        @inbounds for i in 1:(numb_generations + 1)
            res_a[:, i], res_c[:, i] = pop_upd_c_a[:, 1, i], pop_upd_c_a[:, 2, i]
        end

        writedlm("data/agent_type$k.txt", res_a)
        writedlm("data/c_value$k.txt", res_c)
        writedlm("data/fit$k.txt", pop_upd_f)

        population_start = nothing
        pop_start = nothing
        pop_upd_c_a = nothing
        pop_upd_f = nothing
        res_a = nothing
        res_c = nothing
        GC.gc()

        k += 1
    end
end

In [ ]:
sim_run(survWei) # or `sim_run(survGam)` for the Gamma environment